In [3]:
import torch

from data_processing.prepare_data import prepare_inference_input
from model.transformer import TransformerTimeSeriesModel




In [4]:
n_steps=30

model_input = prepare_inference_input(n_steps,"../data/processed/merged_data.csv")
print(model_input)

tensor([[[ 1.5382,  1.8574,  2.3983, -0.4301,  2.3788,  2.3772,  2.3813,
           2.4017],
         [ 1.5382,  1.8574,  2.4410, -0.8964,  2.4227,  2.4487,  2.4176,
           2.4413],
         [ 1.5382,  1.8574,  2.4650, -0.9139,  2.4505,  2.4784,  2.4464,
           2.4637],
         [ 1.5382,  1.8574,  2.4704, -1.1111,  2.4554,  2.4862,  2.4804,
           2.4686],
         [ 1.5382,  1.8574,  2.4530, -0.9905,  2.4472,  2.4615,  2.4699,
           2.4525],
         [ 1.5382,  1.8574,  2.4557, -0.8929,  2.4466,  2.4671,  2.4576,
           2.4550],
         [ 1.5382,  1.8574,  2.4198, -1.0818,  2.4334,  2.4454,  2.4572,
           2.4217],
         [ 1.5382,  1.8574,  2.3489, -0.2563,  2.3627,  2.3629,  2.3862,
           2.3558],
         [ 1.5382,  1.8574,  2.3713, -1.1223,  2.3561,  2.3768,  2.3535,
           2.3766],
         [ 1.5382,  1.8574,  2.3914, -0.8479,  2.3705,  2.3645,  2.3394,
           2.3953],
         [ 1.5382,  1.8574,  2.3932, -0.8341,  2.3682,  2.3701,  2.392

In [15]:

# Transformer 모델 초기화
input_size = 8
d_model = 512
nhead = 8
num_layers = 6
output_size = 1
model = TransformerTimeSeriesModel(input_size, d_model, nhead, num_layers, output_size)
model.load_state_dict(torch.load("../models/Transformer.pth"))  # 학습된 모델 로드
model.eval()

# 예측 수행
with torch.no_grad():
    prediction = model(model_input)
    print(f"Predicted Momentum: {prediction.item():.4f}")



Predicted Momentum: -0.1693


/var/folders/_l/lg0zz1ln2_z8ct4p275jp5880000gn/T/ipykernel_79048/186690793.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/Tr